In [8]:
"""
SWISS DWELLINGS EDA PIPELINE - UPDATED FOR RAW DATA
=============================================================================
Updated to handle raw simulations.csv data without target column
Creates target variable using the same logic as modeling pipeline
=============================================================================
"""

# =============================================================================
# IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import os
import sys
from pathlib import Path
import json
from datetime import datetime
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# =============================================================================
# CONFIGURATION
# =============================================================================

CONFIG = {
    'data_file': 'simulations.csv',
    'sample_size': 100000,  # For EDA, sample to be memory efficient
    'target_col': 'energy_consumption_kwh',  # Will be created
    'key_features': ['layout_area', 'layout_compactness', 'layout_window_perimeter'],
    'save_plots': True,
    'plot_dpi': 150,
    'random_state': 42,
    'output_dirs': {
        'plots': 'eda_output/plots',
        'reports': 'eda_output/reports',
        'data': 'eda_output/data'
    }
}

# Create output directories
for dir_path in CONFIG['output_dirs'].values():
    Path(dir_path).mkdir(parents=True, exist_ok=True)

print("=" * 80)
print("SWISS DWELLINGS EDA PIPELINE - FOR RAW DATASET")
print("=" * 80)
print(f"Configuration:")
print(f"  • Data file: {CONFIG['data_file']}")
print(f"  • Sample size: {CONFIG['sample_size']:,}")
print(f"  • Target column: {CONFIG['target_col']} (will be created)")
print(f"  • Output directories created")
print("=" * 80)

# =============================================================================
# TARGET CREATION FUNCTIONS (From modeling pipeline)
# =============================================================================

def create_realistic_energy_target(df, key_features_list=None):
    """Create realistic energy consumption target (same as modeling pipeline)"""
    print("\n🔧 Creating energy consumption target...")
    
    np.random.seed(CONFIG['random_state'])
    n_samples = len(df)
    
    # Base energy consumption with realistic Swiss distribution
    mu, sigma = 6.5, 0.6  # Parameters for log-normal distribution
    base_energy = np.random.lognormal(mu, sigma, n_samples)
    
    # Scale to realistic Swiss apartment range (kWh/m²/year)
    base_energy = np.interp(base_energy, 
                           (base_energy.min(), base_energy.max()), 
                           (60, 280)).astype('float32')
    
    # Apply feature-based adjustments
    if key_features_list:
        for feature in key_features_list:
            if feature in df.columns:
                values = df[feature].fillna(df[feature].median()).values
                
                if np.std(values) > 0:
                    standardized = (values - np.mean(values)) / np.std(values)
                    standardized = np.clip(standardized, -3, 3) / 3
                else:
                    standardized = np.zeros_like(values)
                
                # Feature-specific impact coefficients
                impact_coefficient = 0
                
                if 'area' in feature.lower():
                    impact_coefficient = 35
                elif 'compact' in feature.lower():
                    impact_coefficient = -25
                elif 'window' in feature.lower():
                    impact_coefficient = 30
                elif 'perimeter' in feature.lower():
                    impact_coefficient = 25
                elif 'sun' in feature.lower():
                    impact_coefficient = -15
                elif 'view' in feature.lower():
                    impact_coefficient = -10
                elif 'altitude' in feature.lower():
                    impact_coefficient = 20
                elif 'age' in feature.lower() or 'year' in feature.lower():
                    impact_coefficient = 15
                
                if impact_coefficient != 0:
                    base_energy += standardized * impact_coefficient
    
    # Ensure realistic bounds
    base_energy = np.clip(base_energy, 40, 350).astype('float32')
    
    # Convert to annual energy (kWh/year)
    if 'layout_area' in df.columns:
        area = df['layout_area'].fillna(df['layout_area'].median()).values
        area_factor = 1 + 0.2 * np.log1p(area / 50)
        annual_energy = base_energy * area * area_factor
    else:
        annual_energy = base_energy * 80
    
    # Add realistic variation factors
    quality_factor = np.random.normal(1.0, 0.15, n_samples)
    occupancy_factor = np.random.normal(1.0, 0.1, n_samples)
    climate_factor = np.random.choice([0.85, 1.0, 1.15], n_samples, p=[0.3, 0.4, 0.3])
    
    annual_energy = annual_energy * quality_factor * occupancy_factor * climate_factor
    
    # Apply final realistic bounds
    annual_energy = np.clip(annual_energy, 3000, 40000).astype('int32')
    
    # Add some random noise for realism
    noise = np.random.normal(0, annual_energy.std() * 0.05, n_samples)
    annual_energy = (annual_energy + noise).astype('int32')
    annual_energy = np.clip(annual_energy, 3000, 40000)
    
    # Create target series
    target = pd.Series(annual_energy, index=df.index, name=CONFIG['target_col'])
    
    print(f"✅ Target created:")
    print(f"   • Mean: {target.mean():,.0f} kWh/year")
    print(f"   • Std:  {target.std():,.0f} kWh/year")
    print(f"   • Min:  {target.min():,.0f} kWh/year")
    print(f"   • Max:  {target.max():,.0f} kWh/year")
    
    return target

# =============================================================================
# DATA LOADING AND PREPARATION
# =============================================================================

def load_and_prepare_data():
    """Load data and create target variable"""
    print("\n📊 LOADING AND PREPARING DATA")
    print("-" * 60)
    
    filepath = CONFIG['data_file']
    sample_size = CONFIG['sample_size']
    
    # Check if file exists
    if not Path(filepath).exists():
        print(f"❌ File '{filepath}' not found.")
        print("Looking for CSV files in current directory...")
        
        csv_files = list(Path(".").glob("*.csv"))
        if csv_files:
            print(f"Found CSV files: {[f.name for f in csv_files[:5]]}")
            filepath = str(csv_files[0])
            print(f"Using: {filepath}")
        else:
            print("No CSV files found. Creating synthetic dataset...")
            return create_synthetic_dataset()
    
    print(f"Reading: {filepath}")
    
    try:
        # Get total rows for sampling
        total_rows = sum(1 for _ in open(filepath)) - 1  # Exclude header
        
        if total_rows > sample_size:
            print(f"Sampling {sample_size:,} rows from {total_rows:,} total rows...")
            # Use skiprows to sample efficiently
            skip = sorted(np.random.choice(
                range(1, total_rows + 1), 
                total_rows - sample_size, 
                replace=False
            ))
            df = pd.read_csv(filepath, skiprows=skip)
        else:
            df = pd.read_csv(filepath)
        
        print(f"✅ Loaded: {len(df):,} rows × {df.shape[1]} columns")
        
        # Check for layout features (needed for target creation)
        layout_features = ['layout_area', 'layout_compactness', 'layout_window_perimeter']
        missing_layout = [f for f in layout_features if f not in df.columns]
        
        if missing_layout:
            print(f"⚠️  Missing layout features: {missing_layout}")
            print("  These may affect target creation quality")
        
        # Create target variable
        key_features = layout_features + ['building_age' if 'building_age' in df.columns else None]
        key_features = [f for f in key_features if f is not None]
        
        target = create_realistic_energy_target(df, key_features)
        df[CONFIG['target_col']] = target
        
        return df
        
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        print("Creating synthetic dataset...")
        return create_synthetic_dataset()

def create_synthetic_dataset(n_samples=50000):
    """Create synthetic dataset with target"""
    print(f"Creating synthetic dataset ({n_samples:,} samples)...")
    
    np.random.seed(CONFIG['random_state'])
    
    # Create synthetic data similar to your actual data structure
    data = {
        'apartment_id': np.arange(n_samples),
        'layout_area': np.random.lognormal(4.2, 0.35, n_samples).clip(15, 250),
        'layout_compactness': np.random.beta(3, 2, n_samples).clip(0.1, 0.95),
        'layout_window_perimeter': np.random.exponential(4, n_samples).clip(0, 25),
        'layout_number_of_windows': np.random.poisson(3, n_samples).clip(0, 10),
        'building_age': np.random.choice(range(1900, 2024, 10), n_samples),
        'altitude': np.random.exponential(500, n_samples).clip(200, 2500),
        'canton': np.random.choice(['ZH', 'BE', 'VD', 'GE', 'TI', 'AG', 'SG'], n_samples),
        'layout_area_type': np.random.choice(['residential', 'commercial', 'mixed'], n_samples)
    }
    
    # Add some connectivity features (common in your data)
    for prefix in ['connectivity_bathroom', 'connectivity_kitchen', 'connectivity_entrance_door']:
        for suffix in ['max', 'mean', 'median', 'stddev']:
            col_name = f'{prefix}_distance_{suffix}'
            data[col_name] = np.random.exponential(5, n_samples).clip(0, 50)
    
    # Add sun/view features
    for time in ['201806211200', '201803210800']:
        for stat in ['median', 'max', 'mean']:
            col_name = f'sun_{time}_{stat}'
            data[col_name] = np.random.uniform(20, 180, n_samples)
    
    for view in ['sky', 'greenery', 'buildings']:
        for stat in ['mean', 'max']:
            col_name = f'view_{view}_{stat}'
            data[col_name] = np.random.beta(3, 3, n_samples)
    
    df = pd.DataFrame(data)
    
    # Create target
    target = create_realistic_energy_target(df)
    df[CONFIG['target_col']] = target
    
    print(f"✅ Synthetic dataset created: {df.shape[0]:,} rows × {df.shape[1]} columns")
    return df

# =============================================================================
# DATA QUALITY ANALYSIS
# =============================================================================

def analyze_data_quality(df):
    """Perform comprehensive data quality analysis"""
    print("\n📈 DATA QUALITY ANALYSIS")
    print("-" * 60)
    
    print(f"Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    # Memory usage
    memory_mb = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage: {memory_mb:.2f} MB")
    
    # Data types
    print(f"\nData types:")
    dtype_counts = df.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"  {dtype}: {count} columns")
    
    # Missing values
    missing_stats = df.isnull().sum()
    missing_total = missing_stats.sum()
    missing_pct = (missing_total / (df.shape[0] * df.shape[1])) * 100
    
    print(f"\nMissing values:")
    print(f"  Total missing: {missing_total:,}")
    print(f"  Percentage: {missing_pct:.2f}%")
    
    # Top columns with missing values
    missing_cols = missing_stats[missing_stats > 0].sort_values(ascending=False)
    if len(missing_cols) > 0:
        print(f"\nTop 10 columns with missing values:")
        for i, (col, count) in enumerate(missing_cols.head(10).items()):
            pct = (count / len(df)) * 100
            print(f"  {i+1:2d}. {col:30s}: {count:6,} ({pct:5.1f}%)")
    
    # Create visualization
    if len(missing_cols) > 0:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        top_missing = missing_cols.head(15)
        bars = ax.barh(range(len(top_missing)), top_missing.values)
        ax.set_yticks(range(len(top_missing)))
        ax.set_yticklabels(top_missing.index)
        ax.set_xlabel('Missing Count')
        ax.set_title('Top 15 Features with Missing Values')
        ax.invert_yaxis()
        
        # Add percentage labels
        for i, (idx, val) in enumerate(top_missing.items()):
            pct = (val / len(df)) * 100
            ax.text(val + max(top_missing.values)*0.01, i, 
                   f'{pct:.1f}%', va='center', fontsize=9)
        
        plt.tight_layout()
        save_plot(fig, 'missing_data_analysis.png', 'quality')
    
    # Save report
    quality_report = {
        'dataset_shape': list(df.shape),
        'memory_mb': float(memory_mb),
        'missing_total': int(missing_total),
        'missing_percentage': float(missing_pct),
        'columns_with_missing': int(len(missing_cols)),
        'data_types': {str(k): int(v) for k, v in dtype_counts.items()}
    }
    
    report_path = Path(CONFIG['output_dirs']['reports']) / 'data_quality_report.json'
    with open(report_path, 'w') as f:
        json.dump(quality_report, f, indent=2)
    
    print(f"\n✅ Data quality report saved: {report_path}")
    return quality_report

# =============================================================================
# TARGET VARIABLE ANALYSIS
# =============================================================================

def analyze_target_variable(df):
    """Analyze the created target variable"""
    target_col = CONFIG['target_col']
    
    print("\n🎯 TARGET VARIABLE ANALYSIS")
    print("-" * 60)
    
    target = df[target_col]
    
    # Calculate statistics
    stats_dict = {
        'count': len(target),
        'mean': float(target.mean()),
        'median': float(target.median()),
        'std': float(target.std()),
        'min': float(target.min()),
        'max': float(target.max()),
        'q25': float(target.quantile(0.25)),
        'q75': float(target.quantile(0.75)),
        'skewness': float(target.skew()),
        'kurtosis': float(target.kurtosis()),
        'cv': float((target.std() / target.mean()) * 100) if target.mean() != 0 else 0
    }
    
    print(f"Target: {target_col}")
    print(f"  Count: {stats_dict['count']:,}")
    print(f"  Mean: {stats_dict['mean']:,.0f} kWh/year")
    print(f"  Median: {stats_dict['median']:,.0f} kWh/year")
    print(f"  Std Dev: {stats_dict['std']:,.0f} kWh/year")
    print(f"  Range: {stats_dict['min']:,.0f} to {stats_dict['max']:,.0f} kWh/year")
    print(f"  IQR: {stats_dict['q75'] - stats_dict['q25']:,.0f} kWh/year")
    print(f"  Skewness: {stats_dict['skewness']:.2f}")
    print(f"  Kurtosis: {stats_dict['kurtosis']:.2f}")
    print(f"  CV: {stats_dict['cv']:.1f}%")
    
    # Create visualization
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # 1. Histogram
    axes[0, 0].hist(target, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].axvline(stats_dict['mean'], color='red', linestyle='--', 
                      label=f'Mean: {stats_dict["mean"]:,.0f}')
    axes[0, 0].axvline(stats_dict['median'], color='green', linestyle='--', 
                      label=f'Median: {stats_dict["median"]:,.0f}')
    axes[0, 0].set_xlabel('Energy (kWh/year)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Energy Consumption Distribution')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Box plot
    axes[0, 1].boxplot(target, vert=False, patch_artist=True)
    axes[0, 1].set_xlabel('Energy (kWh/year)')
    axes[0, 1].set_title('Box Plot')
    axes[0, 1].grid(True, alpha=0.3, axis='x')
    
    # 3. Q-Q plot
    stats.probplot(target, dist="norm", plot=axes[0, 2])
    axes[0, 2].set_title('Q-Q Plot (Normality Check)')
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. ECDF plot
    sorted_target = np.sort(target)
    ecdf = np.arange(1, len(sorted_target) + 1) / len(sorted_target)
    axes[1, 0].plot(sorted_target, ecdf, 'b-', linewidth=2)
    axes[1, 0].set_xlabel('Energy (kWh/year)')
    axes[1, 0].set_ylabel('Cumulative Probability')
    axes[1, 0].set_title('Empirical Cumulative Distribution')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 5. Log transformation
    if target.min() > 0:
        log_target = np.log1p(target)
        axes[1, 1].hist(log_target, bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
        axes[1, 1].set_xlabel('Log(Energy + 1)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('Log-Transformed Distribution')
        axes[1, 1].grid(True, alpha=0.3)
    else:
        axes[1, 1].text(0.5, 0.5, 'Log transform not applicable\n(min value ≤ 0)', 
                       ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Log Transformation Check')
    
    # 6. Statistics summary
    stats_text = f"""
    Statistical Summary:
    
    Central Tendency:
    • Mean: {stats_dict['mean']:,.0f}
    • Median: {stats_dict['median']:,.0f}
    
    Dispersion:
    • Std Dev: {stats_dict['std']:,.0f}
    • IQR: {stats_dict['q75'] - stats_dict['q25']:,.0f}
    • Range: {stats_dict['max'] - stats_dict['min']:,.0f}
    • CV: {stats_dict['cv']:.1f}%
    
    Shape:
    • Skewness: {stats_dict['skewness']:.2f}
    • Kurtosis: {stats_dict['kurtosis']:.2f}
    
    Percentiles:
    • 25th: {stats_dict['q25']:,.0f}
    • 75th: {stats_dict['q75']:,.0f}
    • 90th: {target.quantile(0.9):,.0f}
    """
    
    axes[1, 2].text(0.05, 0.95, stats_text, transform=axes[1, 2].transAxes,
                   fontsize=9, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    axes[1, 2].axis('off')
    
    plt.suptitle('Energy Consumption Target Analysis', fontsize=16, y=1.02)
    plt.tight_layout()
    save_plot(fig, 'target_analysis.png', 'target')
    
    # Save statistics
    stats_path = Path(CONFIG['output_dirs']['reports']) / 'target_statistics.json'
    with open(stats_path, 'w') as f:
        json.dump(stats_dict, f, indent=2)
    
    print(f"\n✅ Target analysis saved: {stats_path}")
    return stats_dict

# =============================================================================
# FEATURE ANALYSIS
# =============================================================================

def analyze_features(df, top_n=20):
    """Analyze features and their correlations with target"""
    target_col = CONFIG['target_col']
    
    print("\n🔍 FEATURE ANALYSIS")
    print("-" * 60)
    
    # Select numeric features (excluding target and ID columns)
    numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Exclude target and ID-like columns
    exclude_patterns = ['id', 'ID', 'Id', 'index']
    numeric_features = [f for f in numeric_features 
                       if f != target_col and 
                       not any(pattern in f.lower() for pattern in exclude_patterns)]
    
    print(f"Analyzing {len(numeric_features)} numeric features...")
    
    # Calculate correlations with target
    correlations = {}
    for feature in numeric_features[:200]:  # Limit to first 200 for speed
        if feature in df.columns and target_col in df.columns:
            try:
                corr = df[feature].corr(df[target_col])
                if not np.isnan(corr):
                    correlations[feature] = corr
            except:
                continue
    
    # Sort by absolute correlation
    sorted_correlations = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)
    
    print(f"\nTop {top_n} features by correlation with {target_col}:")
    print("-" * 80)
    for i, (feature, corr) in enumerate(sorted_correlations[:top_n]):
        direction = "+" if corr > 0 else "-"
        strength = "strong" if abs(corr) > 0.5 else "moderate" if abs(corr) > 0.3 else "weak"
        print(f"{i+1:2d}. {feature:40s}: {corr:7.3f} ({strength} {direction})")
    
    # Create correlation visualization
    if len(sorted_correlations) > 5:
        top_features = [feat for feat, _ in sorted_correlations[:10]]
        top_corrs = [corr for _, corr in sorted_correlations[:10]]
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        colors = ['lightcoral' if c > 0 else 'lightblue' for c in top_corrs]
        bars = ax.barh(range(len(top_features)), top_corrs, color=colors)
        ax.set_yticks(range(len(top_features)))
        ax.set_yticklabels(top_features)
        ax.set_xlabel('Correlation with Energy Consumption')
        ax.set_title('Top 10 Correlated Features')
        ax.invert_yaxis()
        ax.grid(True, alpha=0.3, axis='x')
        
        # Add correlation values
        for i, (bar, corr) in enumerate(zip(bars, top_corrs)):
            ax.text(bar.get_width() + (0.01 if corr > 0 else -0.01), 
                   bar.get_y() + bar.get_height()/2,
                   f'{corr:.3f}', 
                   ha='left' if corr > 0 else 'right', 
                   va='center',
                   fontsize=9)
        
        plt.tight_layout()
        save_plot(fig, 'feature_correlations.png', 'features')
    
    # Analyze key layout features
    print(f"\n📐 Key Layout Features Analysis:")
    print("-" * 60)
    
    key_features = ['layout_area', 'layout_compactness', 'layout_window_perimeter', 
                   'layout_number_of_windows', 'building_age', 'altitude']
    
    for feature in key_features:
        if feature in df.columns:
            if feature in correlations:
                corr = correlations[feature]
                direction = "positive" if corr > 0 else "negative"
                print(f"  • {feature:25s}: r = {corr:.3f} ({direction} correlation)")
            else:
                print(f"  • {feature:25s}: Not in correlation analysis")
        else:
            print(f"  • {feature:25s}: Not found in dataset")
    
    # Save correlations
    corr_results = {
        'top_correlations': {k: float(v) for k, v in sorted_correlations[:50]},
        'total_features_analyzed': len(correlations)
    }
    
    corr_path = Path(CONFIG['output_dirs']['reports']) / 'feature_correlations.json'
    with open(corr_path, 'w') as f:
        json.dump(corr_results, f, indent=2)
    
    print(f"\n✅ Feature analysis saved: {corr_path}")
    return correlations

# =============================================================================
# DATASET EXPLORATION
# =============================================================================

def explore_dataset_structure(df):
    """Explore the structure of the dataset"""
    print("\n🔎 DATASET STRUCTURE EXPLORATION")
    print("-" * 60)
    
    print(f"Total columns: {df.shape[1]}")
    print(f"Total rows: {df.shape[0]:,}")
    
    # Analyze column patterns
    print(f"\nColumn patterns found:")
    
    # Group columns by prefix
    column_prefixes = {}
    for col in df.columns:
        parts = col.split('_')
        if len(parts) > 1:
            prefix = '_'.join(parts[:2])  # First two parts as prefix
            column_prefixes[prefix] = column_prefixes.get(prefix, 0) + 1
    
    # Sort and display top prefixes
    sorted_prefixes = sorted(column_prefixes.items(), key=lambda x: x[1], reverse=True)
    
    print(f"\nTop column groups (by prefix):")
    for prefix, count in sorted_prefixes[:15]:
        print(f"  • {prefix:25s}: {count:3d} columns")
    
    # Show sample of different column types
    print(f"\nSample columns from each major group:")
    
    sample_cols = {}
    for prefix, count in sorted_prefixes[:10]:
        cols_with_prefix = [col for col in df.columns if col.startswith(prefix)]
        sample_cols[prefix] = cols_with_prefix[:3]  # First 3 columns
    
    for prefix, sample in sample_cols.items():
        print(f"\n  {prefix}:")
        for col in sample:
            dtype = str(df[col].dtype)
            unique = df[col].nunique()
            missing = df[col].isnull().sum()
            print(f"    - {col} ({dtype}, {unique} unique, {missing} missing)")
    
    return column_prefixes

# =============================================================================
# MAIN EDA PIPELINE
# =============================================================================

def run_eda_pipeline():
    """Main EDA pipeline execution"""
    print("\n" + "="*80)
    print("SWISS DWELLINGS - EDA PIPELINE EXECUTION")
    print("="*80)
    
    start_time = datetime.now()
    
    try:
        # 1. Load and prepare data
        print("\n1. LOADING AND PREPARING DATA")
        df = load_and_prepare_data()
        
        # 2. Data quality analysis
        print("\n2. DATA QUALITY ANALYSIS")
        quality_report = analyze_data_quality(df)
        
        # 3. Explore dataset structure
        print("\n3. DATASET STRUCTURE EXPLORATION")
        column_prefixes = explore_dataset_structure(df)
        
        # 4. Target variable analysis
        print("\n4. TARGET VARIABLE ANALYSIS")
        target_stats = analyze_target_variable(df)
        
        # 5. Feature analysis
        print("\n5. FEATURE ANALYSIS")
        correlations = analyze_features(df)
        
        # 6. Save processed data
        print("\n6. SAVING PROCESSED DATA")
        
        # Save full dataset
        full_path = Path(CONFIG['output_dirs']['data']) / 'swiss_dwellings_with_target.csv'
        df.to_csv(full_path, index=False)
        print(f"  • Full dataset: {full_path} ({len(df):,} rows)")
        
        # Save sample for analysis
        sample_size = min(10000, len(df))
        sample_path = Path(CONFIG['output_dirs']['data']) / 'swiss_dwellings_sample.csv'
        df_sample = df.sample(sample_size, random_state=CONFIG['random_state'])
        df_sample.to_csv(sample_path, index=False)
        print(f"  • Sample dataset: {sample_path} ({sample_size:,} rows)")
        
        # Save summary
        end_time = datetime.now()
        execution_time = (end_time - start_time).total_seconds()
        
        summary = {
            'execution_time_seconds': execution_time,
            'dataset_rows': len(df),
            'dataset_columns': df.shape[1],
            'target_created': CONFIG['target_col'] in df.columns,
            'features_analyzed': len(correlations) if correlations else 0,
            'output_files': {
                'plots': len(list(Path(CONFIG['output_dirs']['plots']).glob('**/*.png'))),
                'reports': len(list(Path(CONFIG['output_dirs']['reports']).glob('*.json'))),
                'data': len(list(Path(CONFIG['output_dirs']['data']).glob('*.csv')))
            }
        }
        
        summary_path = Path(CONFIG['output_dirs']['reports']) / 'eda_summary.json'
        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=2)
        
        print("\n" + "="*80)
        print("✅ EDA PIPELINE COMPLETED SUCCESSFULLY!")
        print("="*80)
        print(f"Execution time: {execution_time:.1f} seconds")
        print(f"Dataset: {len(df):,} rows × {df.shape[1]} columns")
        print(f"Target created: {CONFIG['target_col']} (mean: {target_stats['mean']:,.0f} kWh/year)")
        print(f"\nOutputs created:")
        print(f"  • Plots: {summary['output_files']['plots']} files in {CONFIG['output_dirs']['plots']}/")
        print(f"  • Reports: {summary['output_files']['reports']} files in {CONFIG['output_dirs']['reports']}/")
        print(f"  • Data: {summary['output_files']['data']} files in {CONFIG['output_dirs']['data']}/")
        print("\n" + "="*80)
        
        return df
        
    except Exception as e:
        print(f"\n❌ Error in EDA pipeline: {e}")
        import traceback
        traceback.print_exc()
        return None

# =============================================================================
# EXECUTION
# =============================================================================

if __name__ == "__main__":
    # Run the EDA pipeline
    df_result = run_eda_pipeline()
    
    if df_result is not None:
        print("\n📋 KEY FINDINGS:")
        print("="*60)
        
        # Display key statistics
        if CONFIG['target_col'] in df_result.columns:
            target_mean = df_result[CONFIG['target_col']].mean()
            target_std = df_result[CONFIG['target_col']].std()
            print(f"1. Energy Consumption:")
            print(f"   • Mean: {target_mean:,.0f} kWh/year")
            print(f"   • Std Dev: {target_std:,.0f} kWh/year")
            print(f"   • Range: Swiss realistic (3,000-40,000 kWh/year)")
        
        # Display data quality
        missing_pct = (df_result.isnull().sum().sum() / (df_result.shape[0] * df_result.shape[1])) * 100
        print(f"\n2. Data Quality:")
        print(f"   • Missing values: {missing_pct:.1f}%")
        print(f"   • Features with >50% missing: {sum(df_result.isnull().sum() > len(df_result)*0.5)}")
        
        # Display feature insights
        numeric_cols = df_result.select_dtypes(include=[np.number]).columns.tolist()
        if CONFIG['target_col'] in numeric_cols:
            numeric_cols.remove(CONFIG['target_col'])
        
        print(f"\n3. Feature Insights:")
        print(f"   • Numeric features: {len(numeric_cols)}")
        print(f"   • Layout features present: {sum(1 for f in CONFIG['key_features'] if f in df_result.columns)}")
        
        print("\n" + "="*60)
        print("✅ EDA ready for modeling pipeline!")
        print("="*60)

SWISS DWELLINGS EDA PIPELINE - FOR RAW DATASET
Configuration:
  • Data file: simulations.csv
  • Sample size: 100,000
  • Target column: energy_consumption_kwh (will be created)
  • Output directories created

SWISS DWELLINGS - EDA PIPELINE EXECUTION

1. LOADING AND PREPARING DATA

📊 LOADING AND PREPARING DATA
------------------------------------------------------------
Reading: simulations.csv
Sampling 100,000 rows from 347,583 total rows...
✅ Loaded: 100,000 rows × 367 columns

🔧 Creating energy consumption target...
✅ Target created:
   • Mean: 3,134 kWh/year
   • Std:  773 kWh/year
   • Min:  3,000 kWh/year
   • Max:  40,000 kWh/year

2. DATA QUALITY ANALYSIS

📈 DATA QUALITY ANALYSIS
------------------------------------------------------------
Dataset shape: 100,000 rows × 368 columns
Memory usage: 285.18 MB

Data types:
  float64: 346 columns
  bool: 10 columns
  int64: 9 columns
  object: 2 columns
  int32: 1 columns

Missing values:
  Total missing: 1,558,141
  Percentage: 4.23%

In [6]:
"""
SWISS DWELLINGS DATA PIPELINE - MEMORY OPTIMIZED (FINAL ENHANCED VERSION - COMPLETELY FIXED)
================================================================================
Final enhanced version with all fixes:
1. Realistic target distribution
2. Model comparison (GradientBoosting, RandomForest, Ridge)
3. Detailed feature importance analysis
4. Better missing value handling
5. Categorical feature encoding support
6. Explicit memory cleanup
7. Cross-validation for robust evaluation
8. Variance threshold for feature selection
================================================================================
"""

# ==============================================================================
# IMPORTS
# ==============================================================================
import pandas as pd
import numpy as np
import json
import sys
import time
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import gc

# Sklearn imports
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib

print("=" * 100)
print("SWISS DWELLINGS: FINAL ENHANCED MEMORY-OPTIMIZED PIPELINE (COMPLETELY FIXED)")
print(f"Python {sys.version.split()[0]}")
print(f"Execution started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 100)

# ==============================================================================
# CONFIGURATION
# ==============================================================================

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', 100)
np.random.seed(42)

# Create output directories
output_dirs = ["output", "output/plots", "output/models", "output/data", "output/reports", "output/cv_results", "output/model_comparison"]
for dir_path in output_dirs:
    Path(dir_path).mkdir(exist_ok=True)

# ==============================================================================
# ENHANCED HELPER FUNCTIONS
# ==============================================================================

def convert_to_json_serializable(obj):
    """Convert numpy and pandas types to JSON serializable Python types."""
    if isinstance(obj, (np.integer, np.int64, np.int32, np.int16, np.int8)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32, np.float16)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, pd.Series):
        return obj.tolist()
    elif isinstance(obj, pd.DataFrame):
        return obj.to_dict(orient='records')
    elif isinstance(obj, dict):
        return {key: convert_to_json_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_json_serializable(item) for item in obj]
    elif isinstance(obj, (datetime, pd.Timestamp)):
        return obj.isoformat()
    elif pd.isna(obj):
        return None
    else:
        return obj

def optimize_memory(df, categorical_threshold=0.5):
    """Optimize DataFrame memory usage by downcasting dtypes with enhanced handling"""
    print(f"  Optimizing memory usage...")
    
    initial_memory = df.memory_usage(deep=True).sum() / 1024**2
    
    # Downcast numeric columns
    for col in df.select_dtypes(include=['int64', 'int32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    
    for col in df.select_dtypes(include=['float64', 'float32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    
    # Enhanced categorical conversion
    for col in df.select_dtypes(include=['object']).columns:
        num_unique = df[col].nunique()
        num_total = len(df[col])
        if num_unique / num_total < categorical_threshold:
            df[col] = df[col].astype('category')
        else:
            # For high cardinality, keep as string but optimize
            df[col] = df[col].astype('string')
    
    final_memory = df.memory_usage(deep=True).sum() / 1024**2
    reduction = (initial_memory - final_memory) / initial_memory * 100
    
    print(f"  Memory reduced from {initial_memory:.2f} MB to {final_memory:.2f} MB ({reduction:.1f}% reduction)")
    
    return df

def clean_memory(*objects):
    """Explicitly clean objects from memory"""
    for obj in objects:
        del obj
    gc.collect()

# ==============================================================================
# 1. ENHANCED DATA LOADING WITH FILE CHECK
# ==============================================================================

def load_data_memory_efficient(filepath='simulations.csv', nrows=None, chunksize=50000):
    """Load data in chunks for memory efficiency with file existence check"""
    print(f"\n{'='*60}")
    print("MEMORY-EFFICIENT DATA LOADING")
    print(f"{'='*60}")
    
    # Check if file exists
    file_path = Path(filepath)
    if not file_path.exists():
        print(f"⚠️  File '{filepath}' not found.")
        print(f"Creating synthetic dataset for demonstration...")
        return create_synthetic_dataset()
    
    try:
        print(f"Reading file structure from {filepath}...")
        sample = pd.read_csv(filepath, nrows=1000)
        columns = sample.columns.tolist()
        print(f"Detected {len(columns)} columns")
        
        # Enhanced dtype detection
        dtype_dict = {}
        for col in columns:
            col_type = sample[col].dtype
            if col_type == 'object':
                # Check if it's actually numeric
                try:
                    pd.to_numeric(sample[col], errors='raise')
                    dtype_dict[col] = 'float32'
                except:
                    # Check cardinality for categorical
                    num_unique = sample[col].nunique()
                    if num_unique / len(sample[col]) < 0.5:
                        dtype_dict[col] = 'category'
                    else:
                        dtype_dict[col] = 'string'
            elif col_type in ['int64', 'int32', 'int16']:
                dtype_dict[col] = 'int32'
            elif col_type in ['float64', 'float32']:
                dtype_dict[col] = 'float32'
            else:
                dtype_dict[col] = col_type
        
        print(f"Using optimized dtypes for memory efficiency")
        
        # Load data
        if nrows:
            print(f"Reading first {nrows:,} rows...")
            df = pd.read_csv(filepath, nrows=nrows, dtype=dtype_dict, low_memory=True)
        else:
            print(f"Reading full file...")
            df = pd.read_csv(filepath, dtype=dtype_dict, low_memory=True)
        
        print(f"\n✅ Loaded dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
        
        # Optimize memory further
        df = optimize_memory(df)
        
        # Print memory usage
        memory_mb = df.memory_usage(deep=True).sum() / 1024**2
        print(f"Memory usage: {memory_mb:.2f} MB")
        
        return df
        
    except Exception as e:
        print(f"❌ ERROR loading data: {e}")
        print("Creating synthetic dataset for demonstration...")
        return create_synthetic_dataset()

def create_synthetic_dataset(n_samples=100000):
    """Create synthetic dataset for demonstration"""
    print("Creating synthetic Swiss dwellings dataset...")
    
    np.random.seed(42)
    
    data = {
        'apartment_id': np.arange(n_samples),
        'layout_area': np.random.lognormal(4.2, 0.35, n_samples).clip(15, 250).astype('float32'),
        'layout_compactness': np.random.beta(3, 2, n_samples).clip(0.1, 0.95).astype('float32'),
        'layout_window_perimeter': np.random.exponential(4, n_samples).clip(0, 25).astype('float32'),
        'layout_number_of_windows': np.random.poisson(3, n_samples).clip(0, 10).astype('int8'),
        'layout_room_count': np.random.choice([1, 2, 3, 4, 5, 6], n_samples, p=[0.05, 0.25, 0.4, 0.2, 0.08, 0.02]).astype('int8'),
        'sun_201806211200_median': np.random.uniform(20, 180, n_samples).astype('float32'),
        'view_sky_mean': np.random.beta(5, 2, n_samples).astype('float32'),
        'view_greenery_mean': np.random.beta(3, 3, n_samples).astype('float32'),
        'building_age': np.random.choice(range(1900, 2024, 10), n_samples).astype('int16'),
        'altitude': np.random.exponential(500, n_samples).clip(200, 2500).astype('float32'),
        'canton': np.random.choice(['ZH', 'BE', 'VD', 'GE', 'TI', 'AG', 'SG'], n_samples).astype('category'),
        'layout_area_type': np.random.choice(['residential', 'commercial', 'mixed'], n_samples).astype('category')
    }
    
    df = pd.DataFrame(data)
    print(f"Created synthetic dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
    return df

# ==============================================================================
# 2. ENHANCED APARTMENT AGGREGATION WITH CATEGORICAL HANDLING
# ==============================================================================

def aggregate_to_apartments_enhanced(df):
    """Enhanced aggregation with categorical feature handling"""
    print(f"\n{'='*60}")
    print("ENHANCED APARTMENT AGGREGATION")
    print(f"{'='*60}")
    
    if 'apartment_id' not in df.columns:
        print("No apartment_id found. Using rows as apartments.")
        return df
    
    print("Aggregating by apartment_id...")
    
    # Group by apartment_id
    grouped = df.groupby('apartment_id')
    
    # Aggregate numeric columns with mean
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'apartment_id' in numeric_cols:
        numeric_cols.remove('apartment_id')
    
    numeric_agg = grouped[numeric_cols].mean()
    
    # Aggregate categorical columns with mode
    categorical_cols = df.select_dtypes(include=['category', 'object', 'string']).columns.tolist()
    if 'apartment_id' in categorical_cols:
        categorical_cols.remove('apartment_id')
    
    categorical_agg = {}
    for col in categorical_cols:
        try:
            categorical_agg[col] = grouped[col].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
        except:
            categorical_agg[col] = grouped[col].agg(lambda x: x.iloc[0])
    
    # Combine results
    apartment_df = numeric_agg.copy()
    for col, series in categorical_agg.items():
        apartment_df[col] = series
    
    apartment_df = apartment_df.reset_index()
    
    print(f"✅ Aggregated to {len(apartment_df):,} apartments")
    print(f"  - Numeric features: {len(numeric_cols)}")
    print(f"  - Categorical features: {len(categorical_cols)}")
    print(f"Final shape: {apartment_df.shape}")
    
    return apartment_df

# ==============================================================================
# 3. ENHANCED FEATURE ENGINEERING WITH CATEGORICAL ENCODING
# ==============================================================================

def engineer_features_enhanced(df):
    """Enhanced feature engineering with categorical encoding"""
    print(f"\n{'='*60}")
    print("ENHANCED FEATURE ENGINEERING")
    print(f"{'='*60}")
    
    df_engineered = df.copy()
    created_features = []
    
    # 1. Create area-to-window ratio
    if 'layout_area' in df.columns and 'layout_window_perimeter' in df.columns:
        df_engineered['area_window_ratio'] = (df_engineered['layout_area'] / 
                                            (df_engineered['layout_window_perimeter'] + 0.001)).astype('float32')
        created_features.append('area_window_ratio')
        print("  Created: area_window_ratio")
    
    # 2. Create solar efficiency score
    if 'sun_201806211200_median' in df.columns and 'layout_window_perimeter' in df.columns:
        df_engineered['solar_efficiency'] = (df_engineered['sun_201806211200_median'] * 
                                           df_engineered['layout_window_perimeter']).astype('float32')
        created_features.append('solar_efficiency')
        print("  Created: solar_efficiency")
    
    # 3. Create compactness efficiency score
    if 'layout_compactness' in df.columns:
        df_engineered['compactness_squared'] = (df_engineered['layout_compactness'] ** 2).astype('float32')
        created_features.append('compactness_squared')
        print("  Created: compactness_squared")
    
    # 4. Create Swiss-specific features
    if 'altitude' in df.columns:
        # Energy impact of altitude (higher = colder = more heating)
        df_engineered['altitude_energy_factor'] = (df_engineered['altitude'] / 1000).astype('float32')
        created_features.append('altitude_energy_factor')
        print("  Created: altitude_energy_factor")
    
    # 5. Age-based features
    if 'building_age' in df.columns:
        current_year = 2024
        df_engineered['building_age_years'] = (current_year - df_engineered['building_age']).astype('int16')
        df_engineered['is_modern_building'] = (df_engineered['building_age_years'] < 30).astype('int8')
        created_features.extend(['building_age_years', 'is_modern_building'])
        print("  Created: building_age_years, is_modern_building")
    
    print(f"\n✅ Created {len(created_features)} engineered features")
    print(f"Total features now: {df_engineered.shape[1]}")
    
    return df_engineered, created_features

# ==============================================================================
# 4. IMPROVED REALISTIC ENERGY TARGET CREATION
# ==============================================================================

def create_improved_realistic_energy_target(df, key_features_list=None):
    """Create improved realistic energy consumption target with proper distribution"""
    print(f"\n{'='*60}")
    print("CREATING IMPROVED REALISTIC ENERGY TARGET")
    print(f"{'='*60}")
    
    np.random.seed(42)
    n_samples = len(df)
    
    # Create more realistic distribution with better spread
    # Use a mix of normal and log-normal distributions
    
    # Base distribution - mixture of normal distributions for different building types
    # Modern buildings (30%): more efficient
    modern_mask = np.random.random(n_samples) < 0.3
    # Old buildings (40%): less efficient  
    old_mask = np.random.random(n_samples) < 0.4
    # Standard buildings (30%): average efficiency
    standard_mask = ~modern_mask & ~old_mask
    
    # Different base energy distributions for each type
    base_energy = np.zeros(n_samples)
    
    # Modern buildings: 60-120 kWh/m²/year
    if modern_mask.any():
        modern_energy = np.random.normal(90, 15, modern_mask.sum())
        modern_energy = np.clip(modern_energy, 60, 120)
        base_energy[modern_mask] = modern_energy
    
    # Old buildings: 120-250 kWh/m²/year
    if old_mask.any():
        old_energy = np.random.lognormal(4.8, 0.3, old_mask.sum())
        old_energy = np.clip(old_energy, 120, 250)
        base_energy[old_mask] = old_energy
    
    # Standard buildings: 80-180 kWh/m²/year
    if standard_mask.any():
        standard_energy = np.random.normal(130, 25, standard_mask.sum())
        standard_energy = np.clip(standard_energy, 80, 180)
        base_energy[standard_mask] = standard_energy
    
    # Add feature-based adjustments with more impact
    if key_features_list:
        for feature in key_features_list:
            if feature in df.columns:
                values = df[feature].fillna(df[feature].median()).values
                
                if np.std(values) > 0:
                    # Standardize and scale to have meaningful impact
                    standardized = (values - np.mean(values)) / np.std(values)
                    # Limit extreme values
                    standardized = np.clip(standardized, -2, 2)
                else:
                    standardized = np.zeros_like(values)
                
                # Feature-specific impact coefficients - increased for more variation
                impact_coefficient = 0
                
                if 'area' in feature.lower():
                    impact_coefficient = 50  # Larger area = more energy
                elif 'compact' in feature.lower():
                    impact_coefficient = -30  # More compact = better insulation
                elif 'window' in feature.lower():
                    impact_coefficient = 40   # More windows = more heat loss
                elif 'perimeter' in feature.lower():
                    impact_coefficient = 35   # Larger perimeter = more surface area
                elif 'sun' in feature.lower():
                    impact_coefficient = -25  # More sun = passive solar gain
                elif 'view' in feature.lower():
                    impact_coefficient = -15  # Better views might correlate with better design
                elif 'altitude' in feature.lower():
                    impact_coefficient = 30   # Higher altitude = colder = more heating
                elif 'age' in feature.lower() or 'year' in feature.lower():
                    impact_coefficient = 25   # Older buildings = less efficient
                
                if impact_coefficient != 0:
                    base_energy += standardized * impact_coefficient
    
    # Ensure realistic bounds
    base_energy = np.clip(base_energy, 40, 350).astype('float32')
    
    # Convert to annual energy (kWh/year)
    if 'layout_area' in df.columns:
        area = df['layout_area'].fillna(df['layout_area'].median()).values
        # Non-linear area efficiency
        area_factor = 0.9 + 0.3 * np.log1p(area / 100)  # Efficiency curve
        annual_energy = base_energy * area * area_factor
    else:
        annual_energy = base_energy * 80
    
    # Add realistic variation
    # Building quality variation (25%)
    quality_factor = np.random.normal(1.0, 0.125, n_samples)
    # Occupancy variation (15%)
    occupancy_factor = np.random.normal(1.0, 0.075, n_samples)
    # Regional climate variation
    climate_factor = np.random.choice([0.85, 1.0, 1.15], n_samples, p=[0.2, 0.6, 0.2])
    
    annual_energy = annual_energy * quality_factor * occupancy_factor * climate_factor
    
    # Apply final realistic bounds
    annual_energy = np.clip(annual_energy, 3000, 40000).astype('int32')
    
    # Add some random noise for realism
    noise = np.random.normal(0, annual_energy.std() * 0.05, n_samples)
    annual_energy = (annual_energy + noise).astype('int32')
    annual_energy = np.clip(annual_energy, 3000, 40000)
    
    # Create target series
    target = pd.Series(annual_energy, index=df.index, name='energy_consumption_kwh')
    
    print(f"✅ IMPROVED REALISTIC Target Statistics:")
    print(f"  Mean: {target.mean():,.0f} kWh/year")
    print(f"  Std:  {target.std():,.0f} kWh/year")
    print(f"  Min:  {target.min():,.0f} kWh/year")
    print(f"  Max:  {target.max():,.0f} kWh/year")
    print(f"  25th percentile: {np.percentile(target, 25):,.0f} kWh/year")
    print(f"  Median: {np.percentile(target, 50):,.0f} kWh/year")
    print(f"  75th percentile: {np.percentile(target, 75):,.0f} kWh/year")
    print(f"  Coefficient of Variation: {(target.std()/target.mean()*100):.1f}%")
    
    # Check if distribution is realistic (not all at minimum)
    unique_values = target.nunique()
    if unique_values < 100:
        print(f"⚠️  Warning: Only {unique_values} unique values in target")
    
    # Create distribution plot
    create_target_distribution_plot(target)
    
    return target

def create_target_distribution_plot(target):
    """Create target distribution visualization"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Histogram
    axes[0, 0].hist(target, bins=50, edgecolor='black', alpha=0.7, color='skyblue')
    axes[0, 0].axvline(target.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {target.mean():,.0f}')
    axes[0, 0].axvline(np.median(target), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(target):,.0f}')
    axes[0, 0].set_xlabel('Annual Energy Consumption (kWh/year)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Energy Consumption Distribution')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Box plot
    axes[0, 1].boxplot(target, vert=False)
    axes[0, 1].set_xlabel('Annual Energy Consumption (kWh/year)')
    axes[0, 1].set_title('Energy Consumption Statistics')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. ECDF plot
    sorted_data = np.sort(target)
    ecdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    axes[1, 0].plot(sorted_data, ecdf, 'b-', linewidth=2)
    axes[1, 0].set_xlabel('Annual Energy Consumption (kWh/year)')
    axes[1, 0].set_ylabel('Cumulative Probability')
    axes[1, 0].set_title('Empirical Cumulative Distribution Function')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Statistics summary
    stats_text = f"""
    Statistical Summary:
    
    Basic Statistics:
    • Mean: {target.mean():,.0f} kWh/year
    • Std Dev: {target.std():,.0f} kWh/year
    • CV: {(target.std()/target.mean()*100):.1f}%
    
    Percentiles:
    • 10th: {np.percentile(target, 10):,.0f} kWh/year
    • 25th: {np.percentile(target, 25):,.0f} kWh/year
    • 50th: {np.percentile(target, 50):,.0f} kWh/year
    • 75th: {np.percentile(target, 75):,.0f} kWh/year
    • 90th: {np.percentile(target, 90):,.0f} kWh/year
    
    Distribution:
    • Unique values: {target.nunique()}
    • Range: {target.max() - target.min():,.0f} kWh/year
    • IQR: {np.percentile(target, 75) - np.percentile(target, 25):,.0f} kWh/year
    """
    
    axes[1, 1].text(0.05, 0.95, stats_text, transform=axes[1, 1].transAxes,
                   fontsize=10, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    axes[1, 1].axis('off')
    
    plt.suptitle('Swiss Dwellings - Improved Realistic Energy Consumption Analysis', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig('output/plots/energy_distribution_improved.png', dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"\n✅ Improved distribution plot saved to output/plots/energy_distribution_improved.png")

# ==============================================================================
# 5. ENHANCED FEATURE SELECTION WITH VARIANCE THRESHOLD
# ==============================================================================

def select_features_enhanced(df, target, max_features=20, variance_threshold=0.01):
    """Enhanced feature selection with variance threshold"""
    print(f"\n{'='*60}")
    print("ENHANCED FEATURE SELECTION")
    print(f"{'='*60}")
    
    # Remove low-variance features first
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df_numeric = df[numeric_cols].copy()
    
    # Apply variance threshold
    variances = df_numeric.var()
    high_variance_cols = variances[variances > variance_threshold].index.tolist()
    
    print(f"Variance filtering: {len(numeric_cols)} → {len(high_variance_cols)} features")
    
    # Calculate correlations for high-variance features
    correlations = {}
    for col in high_variance_cols:
        try:
            vals = df[col].fillna(df[col].median()).values
            if np.std(vals) > 0:
                corr = np.corrcoef(vals, target.values)[0, 1]
                if not np.isnan(corr):
                    correlations[col] = abs(corr)
        except:
            continue
    
    # Sort by absolute correlation
    sorted_correlations = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
    
    # Select top features
    selected_features = [col for col, corr in sorted_correlations[:max_features]]
    
    print(f"✅ Selected {len(selected_features)} features:")
    for i, (col, corr) in enumerate(sorted_correlations[:10]):
        direction = "+" if corr > 0 else "-"
        print(f"  {i+1:2d}. {col:40s}: {corr:6.3f} ({direction})")
    
    # Include important categorical features (but exclude apartment_id for modeling)
    categorical_cols = df.select_dtypes(include=['category']).columns.tolist()
    
    # Filter out apartment_id from categorical features for modeling
    categorical_features_for_modeling = [col for col in categorical_cols if col != 'apartment_id']
    
    if categorical_cols:
        print(f"\nCategorical features available: {', '.join(categorical_cols[:5])}")
        if categorical_features_for_modeling:
            print(f"  Using for modeling: {categorical_features_for_modeling}")
        else:
            print(f"  No categorical features suitable for modeling (excluding apartment_id)")
    
    # Create correlation visualization
    if len(sorted_correlations) > 5:
        top_features = [col for col, corr in sorted_correlations[:10]]
        top_corrs = [corr for col, corr in sorted_correlations[:10]]
        
        plt.figure(figsize=(10, 6))
        bars = plt.barh(range(len(top_features)), top_corrs, color='lightcoral')
        plt.yticks(range(len(top_features)), top_features)
        plt.xlabel('Absolute Correlation with Energy Consumption')
        plt.title('Top 10 Features by Correlation (Variance Filtered)')
        plt.grid(True, alpha=0.3, axis='x')
        
        for i, (bar, corr) in enumerate(zip(bars, top_corrs)):
            plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                    f'{corr:.3f}', ha='left', va='center', fontsize=9)
        
        plt.tight_layout()
        plt.savefig('output/plots/feature_correlations_enhanced.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        print(f"\n✅ Correlation plot saved to output/plots/feature_correlations_enhanced.png")
    
    return selected_features, categorical_features_for_modeling

# ==============================================================================
# 6. MODEL COMPARISON PIPELINE
# ==============================================================================

def build_model_comparison(numeric_features, categorical_features=None):
    """Build multiple models for comparison"""
    print(f"\n{'='*60}")
    print("BUILDING MODEL COMPARISON PIPELINE")
    print(f"{'='*60}")
    
    # Define transformers
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    transformers = [('numeric', numeric_transformer, numeric_features)]
    
    if categorical_features:
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])
        transformers.append(('categorical', categorical_transformer, categorical_features))
        print(f"  Including {len(categorical_features)} categorical features")
    
    preprocessor = ColumnTransformer(transformers=transformers)
    
    # Define models with different characteristics
    models = {
        'GradientBoosting': GradientBoostingRegressor(
            n_estimators=150, 
            learning_rate=0.1, 
            max_depth=4,
            min_samples_split=10, 
            min_samples_leaf=5,
            random_state=42,
            subsample=0.8
        ),
        'RandomForest': RandomForestRegressor(
            n_estimators=100, 
            max_depth=10,
            min_samples_split=10, 
            min_samples_leaf=5,
            random_state=42, 
            n_jobs=-1
        ),
        'Ridge': Ridge(
            alpha=1.0, 
            random_state=42
        )
    }
    
    # Create pipelines
    pipelines = {}
    for name, model in models.items():
        pipelines[name] = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
    
    # Feature summary
    feature_summary = f"{len(numeric_features)} numeric"
    if categorical_features:
        feature_summary += f" + {len(categorical_features)} categorical"
    
    print(f"✅ Model comparison configured with {feature_summary} features")
    print(f"Models: {', '.join(pipelines.keys())}")
    
    return pipelines

def compare_models(pipelines, X_train, X_test, y_train, y_test, feature_names, y_test_actual=None):
    """Compare multiple models and select the best one"""
    print(f"\n{'='*60}")
    print("MODEL COMPARISON EVALUATION")
    print(f"{'='*60}")
    
    results = {}
    
    for name, pipeline in pipelines.items():
        print(f"\n{'─'*40}")
        print(f"Evaluating {name}...")
        print(f"{'─'*40}")
        
        # Perform cross-validation
        try:
            cv_scores = cross_val_score(pipeline, X_train, y_train, 
                                       cv=KFold(n_splits=5, shuffle=True, random_state=42),
                                       scoring='r2',
                                       n_jobs=-1)
            
            print(f"Cross-validation scores: {cv_scores}")
            print(f"Mean CV R²: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
        except Exception as e:
            print(f"⚠️  Cross-validation failed: {e}")
            cv_scores = np.array([0, 0, 0, 0, 0])
        
        # Train final model
        try:
            pipeline.fit(X_train, y_train)
            
            # Make predictions
            y_pred_train = pipeline.predict(X_train)
            y_pred_test = pipeline.predict(X_test)
            
            # Calculate metrics
            metrics = {
                'train_r2': float(r2_score(y_train, y_pred_train)),
                'test_r2': float(r2_score(y_test, y_pred_test)),
                'train_mae': float(mean_absolute_error(y_train, y_pred_train)),
                'test_mae': float(mean_absolute_error(y_test, y_pred_test)),
                'train_rmse': float(np.sqrt(mean_squared_error(y_train, y_pred_train))),
                'test_rmse': float(np.sqrt(mean_squared_error(y_test, y_pred_test))),
                'train_mape': float(np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100),
                'test_mape': float(np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100),
                'cv_mean_r2': float(cv_scores.mean()),
                'cv_std_r2': float(cv_scores.std())
            }
            
            print(f"Test R²: {metrics['test_r2']:.4f}")
            print(f"Test MAE: {metrics['test_mae']:,.0f} kWh/year")
            print(f"Test MAPE: {metrics['test_mape']:.1f}%")
            
            results[name] = {
                'pipeline': pipeline,
                'metrics': metrics,
                'predictions': {
                    'train': y_pred_train,
                    'test': y_pred_test
                }
            }
            
        except Exception as e:
            print(f"❌ Model training failed: {e}")
            continue
    
    if not results:
        print("❌ No models were successfully trained!")
        return {}, None, None
    
    # Find best model based on test R²
    best_model_name = max(results.keys(), key=lambda x: results[x]['metrics']['test_r2'])
    best_model = results[best_model_name]
    
    print(f"\n{'='*60}")
    print(f"🏆 BEST MODEL: {best_model_name}")
    print(f"{'='*60}")
    print(f"Test R²: {best_model['metrics']['test_r2']:.4f}")
    print(f"Test MAE: {best_model['metrics']['test_mae']:,.0f} kWh/year")
    print(f"CV Mean R²: {best_model['metrics']['cv_mean_r2']:.4f}")
    
    # Create model comparison plot
    try:
        create_model_comparison_plot(results, y_test_actual if y_test_actual is not None else y_test)
    except Exception as e:
        print(f"⚠️  Model comparison plot creation failed: {e}")
    
    # Perform detailed feature importance analysis for tree-based models
    for name in ['GradientBoosting', 'RandomForest']:
        if name in results:
            print(f"\n{'─'*40}")
            print(f"Feature Importance Analysis for {name}")
            print(f"{'─'*40}")
            try:
                importance_df = analyze_feature_importance(results[name]['pipeline'], feature_names, X_train)
                if importance_df is not None:
                    # Save importance analysis
                    importance_df.to_csv(f'output/model_comparison/feature_importance_{name.lower()}.csv', index=False)
                    print(f"✅ Feature importance saved for {name}")
            except Exception as e:
                print(f"⚠️  Feature importance analysis failed for {name}: {e}")
    
    return results, best_model_name, best_model

def create_model_comparison_plot(results, y_test):
    """Create visualization comparing all models"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    models = list(results.keys())
    
    # 1. Test R² comparison
    test_r2_values = [results[m]['metrics']['test_r2'] for m in models]
    cv_r2_values = [results[m]['metrics']['cv_mean_r2'] for m in models]
    
    x = np.arange(len(models))
    width = 0.35
    
    axes[0, 0].bar(x - width/2, test_r2_values, width, label='Test R²', color='skyblue')
    axes[0, 0].bar(x + width/2, cv_r2_values, width, label='CV Mean R²', color='lightcoral')
    axes[0, 0].set_xlabel('Model')
    axes[0, 0].set_ylabel('R² Score')
    axes[0, 0].set_title('Model Performance Comparison (R²)')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(models, rotation=45)
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for i, (test_r2, cv_r2) in enumerate(zip(test_r2_values, cv_r2_values)):
        axes[0, 0].text(i - width/2, test_r2 + 0.01, f'{test_r2:.3f}', 
                       ha='center', va='bottom', fontsize=9)
        axes[0, 0].text(i + width/2, cv_r2 + 0.01, f'{cv_r2:.3f}', 
                       ha='center', va='bottom', fontsize=9)
    
    # 2. Error metrics comparison
    mae_values = [results[m]['metrics']['test_mae'] for m in models]
    mape_values = [results[m]['metrics']['test_mape'] for m in models]
    
    # Normalize for better visualization
    if max(mae_values) > 0:
        mae_norm = [v/max(mae_values) for v in mae_values]
    else:
        mae_norm = [0 for _ in mae_values]
    
    if max(mape_values) > 0:
        mape_norm = [v/max(mape_values) for v in mape_values]
    else:
        mape_norm = [0 for _ in mape_values]
    
    axes[0, 1].bar(x - width/2, mae_norm, width, label='MAE (normalized)', color='lightgreen')
    axes[0, 1].bar(x + width/2, mape_norm, width, label='MAPE (normalized)', color='orange')
    axes[0, 1].set_xlabel('Model')
    axes[0, 1].set_ylabel('Normalized Error')
    axes[0, 1].set_title('Error Metrics Comparison (Normalized)')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(models, rotation=45)
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # Add actual values
    for i, (mae, mape) in enumerate(zip(mae_values, mape_values)):
        axes[0, 1].text(i - width/2, mae_norm[i] + 0.02, f'{mae:.0f}', 
                       ha='center', va='bottom', fontsize=9)
        axes[0, 1].text(i + width/2, mape_norm[i] + 0.02, f'{mape:.1f}%', 
                       ha='center', va='bottom', fontsize=9)
    
    # 3. Actual vs Predicted for best model
    best_model_name = max(results.keys(), key=lambda x: results[x]['metrics']['test_r2'])
    best_result = results[best_model_name]
    y_pred_test = best_result['predictions']['test']
    
    # Plot first 100 samples for clarity
    n_samples = min(100, len(y_pred_test))
    sample_indices = range(n_samples)
    
    axes[1, 0].scatter(sample_indices, y_pred_test[:n_samples], 
                      alpha=0.6, s=30, color='blue', label='Predictions')
    axes[1, 0].scatter(sample_indices, y_test.values[:n_samples],
                      alpha=0.6, s=30, color='red', label='Actual', marker='x')
    axes[1, 0].set_xlabel('Sample Index')
    axes[1, 0].set_ylabel('Energy (kWh/year)')
    axes[1, 0].set_title(f'{best_model_name} - Test Predictions vs Actual (First {n_samples})')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Model characteristics
    model_info = {
        'GradientBoosting': 'Tree ensemble, non-linear, handles interactions',
        'RandomForest': 'Tree ensemble, robust, handles outliers',
        'Ridge': 'Linear, interpretable, fast'
    }
    
    info_text = "Model Characteristics:\n\n"
    for model in models:
        info_text += f"• {model}:\n  {model_info.get(model, 'No description')}\n\n"
    
    axes[1, 1].text(0.05, 0.95, info_text, transform=axes[1, 1].transAxes,
                   fontsize=9, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    axes[1, 1].axis('off')
    
    plt.suptitle('Swiss Dwellings - Model Comparison Analysis', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig('output/model_comparison/model_comparison.png', dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"\n✅ Model comparison plot saved to output/model_comparison/model_comparison.png")

def analyze_feature_importance(pipeline, feature_names, X_train):
    """Analyze feature importance in detail for tree-based models"""
    
    # Check if model has feature_importances_ attribute
    if not hasattr(pipeline.named_steps['model'], 'feature_importances_'):
        print("  Model doesn't support feature importance analysis")
        return None
    
    try:
        importances = pipeline.named_steps['model'].feature_importances_
        
        # Handle transformed feature names
        preprocessor = pipeline.named_steps['preprocessor']
        
        # Get all feature names after preprocessing
        all_feature_names = []
        
        # Get numeric feature names
        if 'numeric' in preprocessor.named_transformers_:
            all_feature_names.extend(feature_names)
        
        # Get categorical feature names if they exist
        if 'categorical' in preprocessor.named_transformers_:
            categorical_transformer = preprocessor.named_transformers_['categorical']
            
            # Check if encoder exists in the pipeline
            if hasattr(categorical_transformer, 'named_steps') and 'encoder' in categorical_transformer.named_steps:
                encoder = categorical_transformer.named_steps['encoder']
                
                # Get categorical feature indices from ColumnTransformer
                for name, transformer, cols in preprocessor.transformers:
                    if name == 'categorical':
                        categorical_features = cols
                        break
                else:
                    categorical_features = []
                
                # Get one-hot encoded feature names
                if hasattr(encoder, 'get_feature_names_out'):
                    if categorical_features:
                        encoded_cat_names = encoder.get_feature_names_out(categorical_features)
                        all_feature_names.extend(encoded_cat_names)
        
        # Ensure we have the right number of feature names
        if len(all_feature_names) > len(importances):
            all_feature_names = all_feature_names[:len(importances)]
        elif len(all_feature_names) < len(importances):
            # Create generic names for missing ones
            for i in range(len(all_feature_names), len(importances)):
                all_feature_names.append(f'feature_{i}')
        
        # Create importance DataFrame
        importance_df = pd.DataFrame({
            'feature': all_feature_names,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        # Plot top features
        plt.figure(figsize=(12, 8))
        top_n = min(20, len(importance_df))
        top_features = importance_df.head(top_n)
        
        plt.barh(range(top_n), top_features['importance'][::-1], color='lightblue', edgecolor='darkblue')
        plt.yticks(range(top_n), top_features['feature'][::-1], fontsize=10)
        plt.xlabel('Feature Importance', fontsize=12)
        plt.title(f'Top {top_n} Most Important Features', fontsize=14)
        plt.grid(True, alpha=0.3, axis='x')
        
        # Add importance values
        for i, (_, row) in enumerate(top_features[::-1].iterrows()):
            plt.text(row['importance'] + 0.001, i, f'{row["importance"]:.4f}', 
                    va='center', fontsize=9)
        
        plt.tight_layout()
        
        # Get model name for filename
        model_name = pipeline.named_steps['model'].__class__.__name__
        plt.savefig(f'output/model_comparison/feature_importance_{model_name}.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        print(f"✅ Top 10 features by importance:")
        for i, row in importance_df.head(10).iterrows():
            print(f"  {i+1:2d}. {row['feature']:40s}: {row['importance']:.4f}")
        
        return importance_df
        
    except Exception as e:
        print(f"  Error in feature importance analysis: {e}")
        return None

# ==============================================================================
# 7. FINAL ENHANCED MAIN EXECUTION PIPELINE (COMPLETELY FIXED)
# ==============================================================================

def run_final_enhanced_analysis():
    """Run final enhanced memory-optimized analysis pipeline - COMPLETELY FIXED"""
    print("=" * 100)
    print("SWISS DWELLINGS - FINAL ENHANCED ANALYSIS PIPELINE")
    print("=" * 100)
    
    start_time = time.time()
    
    try:
        # Step 1: Load data
        print("\n" + "="*60)
        print("STEP 1: LOADING DATA")
        print("="*60)
        
        df = load_data_memory_efficient('simulations.csv', nrows=100000)
        
        # Step 2: Aggregate to apartments
        print("\n" + "="*60)
        print("STEP 2: APARTMENT AGGREGATION")
        print("="*60)
        apartment_df = aggregate_to_apartments_enhanced(df)
        
        # Clean up memory
        clean_memory(df)
        
        # Step 3: Feature engineering
        print("\n" + "="*60)
        print("STEP 3: FEATURE ENGINEERING")
        print("="*60)
        engineered_df, engineered_features = engineer_features_enhanced(apartment_df)
        
        # Clean up memory
        clean_memory(apartment_df)
        
        # Step 4: Identify key features
        print("\n" + "="*60)
        print("STEP 4: IDENTIFYING KEY FEATURES")
        print("="*60)
        
        key_features = []
        feature_patterns = ['area', 'compact', 'window', 'perimeter', 'room', 'sun', 'view', 'altitude', 'age']
        
        for pattern in feature_patterns:
            matching = [col for col in engineered_df.columns if pattern in col.lower()]
            if matching:
                key_features.extend(matching[:2])
        
        key_features = list(set(key_features))
        
        print(f"Identified {len(key_features)} key features:")
        for i, feat in enumerate(key_features[:10], 1):
            print(f"  {i:2d}. {feat}")
        
        # Step 5: Create improved realistic target
        print("\n" + "="*60)
        print("STEP 5: CREATING IMPROVED REALISTIC ENERGY TARGET")
        print("="*60)
        target = create_improved_realistic_energy_target(engineered_df, key_features)
        
        # Step 6: Feature selection
        print("\n" + "="*60)
        print("STEP 6: FEATURE SELECTION")
        print("="*60)
        numeric_features, categorical_features = select_features_enhanced(
            engineered_df, target, max_features=15
        )
        
        # Step 7: Prepare data
        print("\n" + "="*60)
        print("STEP 7: PREPARING DATA FOR MODELING")
        print("="*60)
        
        # Use selected features
        modeling_features = numeric_features + categorical_features
        X = engineered_df[modeling_features].copy()
        y = target  # Store the full target for saving
        
        # Handle missing values with appropriate strategies
        X[numeric_features] = X[numeric_features].fillna(X[numeric_features].median())
        if categorical_features:
            for cat_col in categorical_features:
                mode_values = X[cat_col].mode()
                if not mode_values.empty:
                    X[cat_col] = X[cat_col].fillna(mode_values.iloc[0])
                else:
                    X[cat_col] = X[cat_col].fillna('Unknown')
        
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, shuffle=True
        )
        
        print(f"✅ Data prepared:")
        print(f"  Training set: {len(X_train):,} samples")
        print(f"  Test set:     {len(X_test):,} samples")
        print(f"  Features:     {len(numeric_features)} numeric" + 
              (f", {len(categorical_features)} categorical" if categorical_features else ""))
        
        # Step 8: Build model comparison pipeline
        print("\n" + "="*60)
        print("STEP 8: BUILDING MODEL COMPARISON PIPELINE")
        print("="*60)
        pipelines = build_model_comparison(numeric_features, categorical_features)
        
        # Step 9: Compare models and select best
        print("\n" + "="*60)
        print("STEP 9: MODEL COMPARISON AND SELECTION")
        print("="*60)
        all_results, best_model_name, best_model = compare_models(
            pipelines, X_train, X_test, y_train, y_test, modeling_features, y_test
        )
        
        if not all_results:
            print("❌ No models were successfully trained. Creating basic output...")
            # Create at least some basic output
            create_basic_output(engineered_df, target)
            return None
        
        # Step 10: Save results - PASS THE FULL TARGET 'y'
        print("\n" + "="*60)
        print("STEP 10: SAVING FINAL RESULTS")
        print("="*60)
        
        save_final_results(all_results, best_model_name, best_model, engineered_df, 
                          X_train, X_test, y_train, y_test, y,  # PASS y HERE
                          numeric_features, categorical_features)
        
        # Calculate execution time
        end_time = time.time()
        execution_time = end_time - start_time
        
        # Final summary
        print_final_summary_enhanced(engineered_df, y, numeric_features, 
                                    categorical_features, all_results, 
                                    best_model_name, execution_time)
        
        return {
            'all_results': all_results,
            'best_model': best_model,
            'best_model_name': best_model_name,
            'features': {
                'numeric': numeric_features,
                'categorical': categorical_features
            },
            'data': engineered_df,
            'target': y
        }
        
    except Exception as e:
        print(f"\n❌ ERROR in final enhanced pipeline: {e}")
        import traceback
        traceback.print_exc()
        save_error_report(e, traceback)
        # Create basic output even if pipeline fails
        try:
            if 'engineered_df' in locals() and 'target' in locals():
                create_basic_output(engineered_df, target)
            elif 'engineered_df' in locals():
                create_basic_output(engineered_df, None)
            else:
                create_basic_output(None, None)
        except:
            print("⚠️  Could not create basic output")
        return None

def create_basic_output(engineered_df=None, target=None):
    """Create basic outputs if main pipeline fails"""
    print("\nCreating basic demonstration output...")
    
    try:
        # Create a simple analysis if we have data
        if engineered_df is not None and target is not None:
            # Basic statistics
            stats = {
                'n_apartments': len(engineered_df),
                'target_mean': float(target.mean()),
                'target_std': float(target.std()),
                'target_min': float(target.min()),
                'target_max': float(target.max())
            }
            
            # Save basic report
            basic_report = {
                'timestamp': datetime.now().isoformat(),
                'status': 'basic_output_created',
                'statistics': stats,
                'message': 'Main pipeline failed, basic output created'
            }
            
            with open('output/reports/basic_report.json', 'w') as f:
                json.dump(basic_report, f, indent=2)
            
            print("✅ Basic report saved")
    except Exception as e:
        print(f"⚠️  Could not create basic output: {e}")

def save_final_results(all_results, best_model_name, best_model, engineered_df, 
                      X_train, X_test, y_train, y_test, y_full,  # ADDED y_full parameter
                      numeric_features, categorical_features):
    """Save final enhanced results - FIXED with y_full parameter"""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. Save the best model
    model_data = {
        'model': best_model['pipeline'],
        'model_name': best_model_name,
        'features': {
            'numeric': numeric_features,
            'categorical': categorical_features
        },
        'metrics': best_model['metrics'],
        'training_date': datetime.now().isoformat(),
        'version': f'2.2_{timestamp}',
        'data_summary': {
            'n_samples': len(engineered_df),
            'n_features': len(numeric_features) + len(categorical_features),
            'target_mean': float(y_train.mean()),
            'target_std': float(y_train.std())
        }
    }
    
    if hasattr(best_model['pipeline'].named_steps['model'], 'feature_importances_'):
        model_data['feature_importances'] = best_model['pipeline'].named_steps['model'].feature_importances_.tolist()
    
    best_model_filename = f'output/models/best_model_{best_model_name.lower()}_{timestamp}.pkl'
    joblib.dump(model_data, best_model_filename)
    print(f"✅ Best model saved: {best_model_filename}")
    
    # 2. Save all models comparison results
    comparison_results = {}
    for model_name, result in all_results.items():
        comparison_results[model_name] = {
            'metrics': result['metrics'],
            'model_type': result['pipeline'].named_steps['model'].__class__.__name__
        }
    
    comparison_filename = f'output/model_comparison/model_comparison_results_{timestamp}.json'
    with open(comparison_filename, 'w') as f:
        json.dump(convert_to_json_serializable(comparison_results), f, indent=2)
    print(f"✅ Model comparison results saved: {comparison_filename}")
    
    # 3. Save processed data - FIXED: using y_full instead of undefined y
    processed_data = engineered_df[numeric_features + categorical_features].copy()
    processed_data['energy_consumption_kwh'] = y_full
    processed_filename = f'output/data/swiss_dwellings_processed_final_{timestamp}.csv'
    processed_data.to_csv(processed_filename, index=False)
    print(f"✅ Processed data saved: {processed_filename}")
    
    # 4. Save comprehensive report
    report = {
        'timestamp': datetime.now().isoformat(),
        'project': 'Swiss Dwellings Energy Analysis - Final Enhanced (Completely Fixed)',
        'version': f'2.2_{timestamp}',
        'best_model': best_model_name,
        'best_model_performance': best_model['metrics'],
        'model_comparison': comparison_results,
        'execution_summary': {
            'train_samples': int(len(X_train)),
            'test_samples': int(len(X_test)),
            'features_selected': {
                'numeric': int(len(numeric_features)),
                'categorical': int(len(categorical_features))
            },
            'cross_validation_folds': 5,
            'models_tested': list(all_results.keys())
        },
        'features': {
            'numeric': numeric_features,
            'categorical': categorical_features
        },
        'deliverables': [
            best_model_filename,
            comparison_filename,
            processed_filename,
            'output/plots/energy_distribution_improved.png',
            'output/plots/feature_correlations_enhanced.png',
            'output/model_comparison/model_comparison.png'
        ]
    }
    
    report = convert_to_json_serializable(report)
    report_filename = f'output/reports/final_report_comprehensive_{timestamp}.json'
    with open(report_filename, 'w') as f:
        json.dump(report, f, indent=2)
    
    print(f"✅ Comprehensive report saved: {report_filename}")

def print_final_summary_enhanced(engineered_df, y, numeric_features, 
                                categorical_features, all_results, 
                                best_model_name, execution_time):
    """Print final enhanced summary"""
    best_model_metrics = all_results[best_model_name]['metrics']
    
    print(f"\n" + "="*60)
    print("FINAL ENHANCED RESULTS SUMMARY")
    print("="*60)
    print(f"🏆 Best Model: {best_model_name}")
    print(f"🔹 Model Performance:")
    print(f"   • Test R²: {best_model_metrics['test_r2']:.4f} ({best_model_metrics['test_r2']*100:.1f}% variance explained)")
    print(f"   • CV Mean R²: {best_model_metrics['cv_mean_r2']:.4f} (robust estimate)")
    print(f"   • Test MAE: {best_model_metrics['test_mae']:,.0f} kWh/year")
    print(f"   • Test MAPE: {best_model_metrics['test_mape']:.1f}% error")
    
    print(f"\n🔹 Model Comparison (Test R²):")
    for model_name, result in all_results.items():
        indicator = "🏆" if model_name == best_model_name else "  "
        print(f"   {indicator} {model_name:20s}: {result['metrics']['test_r2']:.4f}")
    
    print(f"\n🔹 Dataset Information:")
    print(f"   • Apartments analyzed: {len(engineered_df):,}")
    print(f"   • Features used: {len(numeric_features)} numeric, {len(categorical_features)} categorical")
    print(f"   • Energy range: {y.min():,.0f} to {y.max():,.0f} kWh/year")
    print(f"   • Average consumption: {y.mean():,.0f} kWh/year")
    
    print(f"\n🔹 Enhanced Features:")
    print(f"   • Improved realistic target distribution created")
    print(f"   • Model comparison performed ({len(all_results)} models)")
    print(f"   • Cross-validation with 5-fold")
    print(f"   • Variance threshold filtering applied")
    print(f"   • Memory optimization with explicit cleanup")
    
    print(f"\n🔹 Swiss Context:")
    print(f"   • Based on Swiss building characteristics")
    print(f"   • Accounts for building age, layout, and regional factors")
    print(f"   • Suitable for Swiss building stock energy analysis")
    
    print(f"\n🔹 Execution Information:")
    print(f"   • Total execution time: {execution_time:.1f} seconds")
    print(f"   • Memory optimized pipeline")
    print(f"   • All outputs saved to 'output/' directory")
    
    print(f"\n" + "="*60)
    
    if best_model_metrics['test_r2'] > 0.7 and best_model_metrics['cv_mean_r2'] > 0.65:
        print("🎉 EXCELLENT: Strong and robust predictive power!")
    elif best_model_metrics['test_r2'] > 0.5:
        print("👍 GOOD: Reliable predictions for Swiss dwellings")
    elif best_model_metrics['test_r2'] > 0.3:
        print("⚠️  MODERATE: Some predictive value")
    else:
        print("🔧 FAIR: Model provides baseline predictions")
    
    print("="*60)

def save_error_report(e, traceback):
    """Save error report"""
    error_report = {
        'error': str(e),
        'timestamp': datetime.now().isoformat(),
        'traceback': traceback.format_exc()
    }
    
    with open('output/reports/error_report_final.json', 'w') as f:
        json.dump(convert_to_json_serializable(error_report), f, indent=2)
    
    print(f"\nError report saved to output/reports/error_report_final.json")

# ==============================================================================
# RUN THE FINAL ENHANCED PIPELINE
# ==============================================================================

if __name__ == "__main__":
    print("\n" + "="*60)
    print("SWISS DWELLINGS - FINAL ENHANCED OPTIMIZED PIPELINE (COMPLETELY FIXED)")
    print("="*60)
    print("Completely fixed issues:")
    print("• Added y_full parameter to save_final_results function")
    print("• Improved realistic target distribution with mixture model")
    print("• Better error handling and basic output creation")
    print("="*60)
    
    start_time = time.time()
    results = run_final_enhanced_analysis()
    end_time = time.time()
    
    if results:
        total_time = end_time - start_time
        print("\n" + "="*60)
        print("✅ FINAL ENHANCED ANALYSIS COMPLETED SUCCESSFULLY")
        print("="*60)
        print(f"Total execution time: {total_time:.1f} seconds")
        print(f"Best Model: {results['best_model_name']}")
        print(f"Best Test R²: {results['best_model']['metrics']['test_r2']:.4f}")
        print(f"Best Test MAE: {results['best_model']['metrics']['test_mae']:,.0f} kWh/year")
        print(f"Features used: {len(results['features']['numeric'])} numeric, {len(results['features']['categorical'])} categorical")
        print(f"\nAll outputs saved to 'output/' directory")
        print("="*60)
    else:
        print("\n" + "="*60)
        print("⚠️  ANALYSIS COMPLETED WITH BASIC OUTPUTS")
        print("="*60)
        print("Check output/reports/error_report_final.json for details")
        print("Basic outputs created in 'output/' directory")
        print("="*60)

SWISS DWELLINGS: FINAL ENHANCED MEMORY-OPTIMIZED PIPELINE (COMPLETELY FIXED)
Python 3.12.6
Execution started: 2025-12-29 15:42:26

SWISS DWELLINGS - FINAL ENHANCED OPTIMIZED PIPELINE (COMPLETELY FIXED)
Completely fixed issues:
• Added y_full parameter to save_final_results function
• Improved realistic target distribution with mixture model
• Better error handling and basic output creation
SWISS DWELLINGS - FINAL ENHANCED ANALYSIS PIPELINE

STEP 1: LOADING DATA

MEMORY-EFFICIENT DATA LOADING
Reading file structure from simulations.csv...
Detected 367 columns
Using optimized dtypes for memory efficiency
Reading first 100,000 rows...

✅ Loaded dataset: 100,000 rows × 367 columns
  Optimizing memory usage...
  Memory reduced from 137.83 MB to 136.31 MB (1.1% reduction)
Memory usage: 136.31 MB

STEP 2: APARTMENT AGGREGATION

ENHANCED APARTMENT AGGREGATION
Aggregating by apartment_id...
✅ Aggregated to 11,913 apartments
  - Numeric features: 355
  - Categorical features: 1
Final shape: (119